# Tutorial de Web Scraping usando BeautifulSoup y Python
Aprende a hacer web scraping de datos usando la biblioteca BeautifulSoup

## Acerca de BeautifulSoup
BeautifulSoup nos facilita analizar gramáticamente datos útiles de un sitio web construido en HTML.
Para poder usar esta biblioteca, tenemos primero que
instalarla en nuestro ambiente local utilizando la terminal y ejecutando el siguiente comando: pip install beautifulsoup4

Puedes echar un vistazo a la [documentación oficial de BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) para ver el conjunto completo de funciones.

### Importamos la biblioteca requests para obtener el contenido HTML.
Puedes ver que nuestro ejemplo contiene cerca de 42k caracteres.

In [64]:
import requests
r = requests.get('https://www.usclimatedata.com/climate/united-states/us')
print(len(r.text))

42402


### Importamos BeautifulSoup y convertimos el HTML en un objeto bs4 

Ahora podemos acceder a etiquetas específicas de HTML en la página usando el punto (.), justo como si fuera un JSON Object.

In [65]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text)
print(soup.title)
print(soup.title.string)


#############################################
# Reto 0. Utiliza otra página web , observa su árbol de etiquetas desde Chrome 
# Dev Tools y usa la documentación de bs4 para imprimir 3 etiquetas más.
#############################################


<title>Climate United States - Normals and averages</title>
Climate United States - Normals and averages



### Profundizando en el objeto bs4 para acceder a contenido del sitio web.

soup.p nos dará el contenido de la primera etiqueta de tipo párrafo en la página web.

soup.a nos da los hipervínculos del sitio.

Podemos obtener los contenidos de un atributo dentro de una etiqueta HTML usando los brackets cuadrados y paréntesis.

Usamos .parent para obtener el objeto padre, y .next_sibling para obtener el objeto en la misma dimension.

**Podemos usar Chrome Dev Tools para inspeccionar elementos y encontrar la etiqueta que necesitemos**


In [66]:
print(soup.p)
print(soup.p.text)
print(soup.a)
print(soup.a['title'])
print()
print(soup.p.parent)

<p class="selection_title">Select a state by name</p>
Select a state by name
<a class="navbar-brand" href="/" title="Temperature - Precipitation - Sunshine - Snowfall"><img alt="Temperature - Precipitation - Sunshine - Snowfall" data-src="https://www.usclimatedata.com/assets/images/us-climate-data.png" height="34" src="https://www.usclimatedata.com/assets/images/us-climate-data.png" srcset="https://www.usclimatedata.com/assets/images/us-climate-data.png 1x, https://www.usclimatedata.com/assets/images/us-climate-data-2.png 2x" width="31"/><span class="white ml-2">U.S. Climate Data</span></a>
Temperature - Precipitation - Sunshine - Snowfall

<div class="float-left mb-4 mt-2"><p class="selection_title">Select a state by name</p></div>


### Prettify() es muy útil para formateo de texto

Este método es genial pero pero obseremos que sólo funciona con objetos bs4, no en caracteres, diccionarios o listas. Para esos objetos necesitamos importar pprint.


In [67]:
print(soup.p.parent.prettify())

<div class="float-left mb-4 mt-2">
 <p class="selection_title">
  Select a state by name
 </p>
</div>



### Necesitamos todos los links de cada estado en esta página
Primero necesitamos encontrar todos los hipervínculos, para imprimir únicamente el atributo href, que es el link realmente.

Pero podemos observar que el resultado incluye algunos hipervínculos que no necesitamos, entonces necesitamos filtrar y remover esos.


In [68]:
for link in soup.find_all('a'):
    print(link.get('href'))

/
#
/
/climate/united-states/us
/
/climate/united-states/us
/climate/alabama/united-states/3170
/climate/alaska/united-states/3171
/climate/arizona/united-states/3172
/climate/arkansas/united-states/3173
/climate/california/united-states/3174
/climate/colorado/united-states/3175
/climate/connecticut/united-states/3176
/climate/delaware/united-states/3177
/climate/district-of-columbia/united-states/3178
/climate/florida/united-states/3179
/climate/georgia/united-states/3180
/climate/hawaii/united-states/3181
/climate/idaho/united-states/3182
/climate/illinois/united-states/3183
/climate/indiana/united-states/3184
/climate/iowa/united-states/3185
/climate/kansas/united-states/3186
/climate/kentucky/united-states/3187
/climate/louisiana/united-states/3188
/climate/maine/united-states/3189
/climate/maryland/united-states/1872
/climate/massachusetts/united-states/3191
/climate/michigan/united-states/3192
/climate/minnesota/united-states/3193
/climate/mississippi/united-states/3194
/climate/

### Filter urls using string functions
### Filtrando URLs usando funciones de caracteres
Con un mágico **if** para verificar condiciones, y luego agregamos los que nos importan a una lista. 
Al final, obtendremos 51 hipervínculos de estado, incluyendo Washington DC.


In [69]:
base_url = 'https://www.usclimatedata.com'
state_links = []
for link in soup.find_all('a'):
    url = link.get('href')
    if url and '/climate/' in url and '/climate/united-states/us' not in url:
        state_links.append(url)
print(len(state_links))

53


### Hagamos una prueba obteniendo datos de sólo un estado
Y luego imprimimos el título de esa página web.

In [70]:
r = requests.get(base_url + state_links[5])
soup = BeautifulSoup(r.text)
print(soup.title.string)

Climate Colorado - Temperature, Rainfall and Averages


### Los datos que necesitamos están en la etiqueta *tr*
Pero mira, hay 14 etiquetas de tipo tr en la página, y sólo queremos 2 de ellas (las columnas de **average high**)


In [76]:
rows = soup.find_all('tr')
print(len(rows))

14


### Filtrando filas, y agregando datos de temperatura a la lista.
Usaremos **List comprehension** para filtrar las filas.
Entonces ahora tendremos sólo 2 filas.
Iteramos esas 2 filasm y agregamos todas las temperaturas para las celdas de datos (td) en la lista.


In [77]:
rows = [row for row in rows if 'Average high' in str(row)]
print(len(rows))

high_temps = []
for row in rows:
    tds = row.find_all('td')
    for i in range(1,2):
        high_temps.append(tds[i].text)
print(high_temps)

2
['58', '91']


### Obteniendo el nombre del estado
Lo que haremos ahora es primero partir la cadena de caracteres en una lista, y tomamos la segunda palabra.
Pero eso no funcionaría para estados con 2 palabras como New York o Carolina del Norte.
Entonces vamos a usar una función de cadenas de caracteres para encontrar y cortar una cadena de caracteres desde el guión (-)


In [79]:
state = soup.title.string.split()[1]
print(state)
s = soup.title.string
state = s[s.find(' '):s.find('-')].strip()
print(state)

Alabama
Alabama


### Add state name and temp list to the data dictionary
### Agregamos el nombre del estado y una lista temporal al diccionario de datos.

Para un estado, así es como se ven nuestros datos con la técnica de scrapping. 
En este ejemplo sólo obtuvimos temperaturas máximas por estado, pero podríamos obtener cada ciuidad, o también obtener otros datos relevantes como las temperaturas mínimas y precipicitación.


In [80]:
data = {}
data[state] = high_temps
print(data)

{'Alabama': ['58', '91']}


### Put it all together and iterate 51 states
### Juntando todo e iterando en 51 estados.
Acá vamos a crear un ciclo en nuestra lista de 51 estados, y obtener la temperatura más alta para cada estado, y agregarla al diccionario de datos.

Esto combina todo nuestro trabajo anterior en un sólo ciclo.
El resultado es un diccionario con 51 estados y una lista de temperaturas máximas para cada uno.


In [82]:
data = {}
for state_link in state_links:
    url = base_url + state_link
    r = requests.get(base_url + state_link)
    soup = BeautifulSoup(r.text)
    rows = soup.find_all('tr')
    rows = [row for row in rows if 'Average high' in str(row)]
    high_temps = []
    for row in rows:
        tds = row.find_all('td')
        for i in range(1,2):
            high_temps.append(tds[i].text)
    s = soup.title.string
    state = s[s.find(' '):s.find('-')].strip()
    data[state] = high_temps
print(data)

{'Alabama': ['58', '91'], 'Alaska': ['27', '64'], 'Arizona': ['71', '104'], 'Arkansas': ['55', '93'], 'California': ['60', '91'], 'Colorado': ['46', '88'], 'Connecticut': ['40', '81'], 'Delaware': ['47', '85'], 'District Of Columbia': ['44', '84'], 'Florida': ['67', '92'], 'Georgia': ['57', '88'], 'Hawaii': ['80', '89'], 'Idaho': ['45', '90'], 'Illinois': ['36', '82'], 'Indiana': ['40', '83'], 'Iowa': ['36', '84'], 'Kansas': ['45', '89'], 'Kentucky': ['45', '86'], 'Louisiana': ['65', '91'], 'Maine': ['32', '78'], 'Maryland': ['46', '87'], 'Massachusetts': ['39', '80'], 'Michigan': ['33', '80'], 'Minnesota': ['31', '82'], 'Mississippi': ['60', '92'], 'Missouri': ['45', '88'], 'Montana': ['39', '85'], 'Nebraska': ['37', '86'], 'Nevada': ['50', '88'], 'New Hampshire': ['35', '81'], 'New Jersey': ['42', '84'], 'New Mexico': ['48', '83'], 'New York': ['42', '83'], 'North Carolina': ['55', '87'], 'North Dakota': ['28', '83'], 'Ohio': ['40', '84'], 'Oklahoma': ['55', '93'], 'Oregon': ['52', '

### Save to CSV file
### Guardando todo en un archivo CSV.
Finalmente, podría interesarnos escribir todo en un archivo CSV.
Acá tenemos una manera fácil para lograr eso.

In [84]:
import csv

with open('high_temps.csv','w') as f:
    w = csv.writer(f)
    w.writerows(data.items())